In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module='h2o')
import pandas.api.types as ptypes
import pickle
from copy import deepcopy
import h2o

In [ ]:
def Project_1_Scoring(testdata):
    holdoutset = testdata.copy()
    artifacts_dict_file = open("../artifacts/artifacts_dict_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()
    artifacts_functions_file = open("../artifacts/artifacts_functions_file.pkl", "rb")
    artifacts_functions = pickle.load(file=artifacts_functions_file)
    artifacts_functions_file.close()
    exec(artifacts_functions["get_naics"], globals())
    exec(artifacts_functions["fillMissingValues"], globals())
    exec(artifacts_functions["data_clean_preprocess"], globals())
    exec(artifacts_functions["create_engineered_features"], globals())
    data_clean_preprocess(holdoutset)
    create_engineered_features(holdoutset,artifacts_dict)
    h2o.init(max_mem_size = "4G",nthreads=4)            
    h2o.remove_all() 
    h2o_hold = h2o.H2OFrame(holdoutset)
    h2o_hold[artifacts_dict["CategoricalColumns"]] = h2o_hold[artifacts_dict["CategoricalColumns"]].asfactor()
    saved_model = h2o.load_model('../'+artifacts_dict["ModelPath"])
    predictions = saved_model.predict(h2o_hold)
    pred_df = predictions.as_data_frame()
    output_df = pd.DataFrame()
    output_df['index']= holdoutset[['index']]
    output_df['label'] = (pred_df['p1'] > artifacts_dict["threshold"]).astype(int) # Converting predictions to integer labels
    output_df['probability_0'] = pred_df['p0']
    output_df['probability_1'] = pred_df['p1']
    return output_df

In [ ]:
testdata = pd.read_csv("../SBA_loans_project_1_holdout_students_valid(1).csv")
scoreddata = Project_1_Scoring(testdata)
scoreddata.head()